In [22]:
import pandas as pd
import numpy as np

care_areas_path = 'CareAreas.csv'
metadata_path = 'metadata.csv'
mainfields_output_path = 'MainFields.csv'
subfields_output_path = 'SubFields.csv'

care_areas_df = pd.read_csv(care_areas_path, header=None)
care_areas_df.columns = ['ID', 'Xmin', 'Xmax', 'Ymin', 'Ymax']
metadata_df = pd.read_csv(metadata_path)

main_field_size = metadata_df['Main Field Size'][0]
sub_field_size = metadata_df['Sub Field size'][0]

care_areas = care_areas_df[['Xmin', 'Xmax', 'Ymin', 'Ymax']].values.tolist()

def is_overlap(rect1, rect2):
    x1, x2, y1, y2 = rect1
    a1, a2, b1, b2 = rect2
    return not (x2 <= a1 or x1 >= a2 or y2 <= b1 or y1 >= b2)

def is_fully_covered(field, care_area):
    fx1, fx2, fy1, fy2 = field
    cx1, cx2, cy1, cy2 = care_area
    return (fx1 <= cx1 and fx2 >= cx2 and fy1 <= cy1 and fy2 >= cy2)

def place_main_fields(main_field_size, care_areas):
    main_fields = []
    covered_areas = []
    main_field_id = 0
    
    for care_area in care_areas:
        x1, x2, y1, y2 = care_area
        x_range = np.arange(x1, x2, main_field_size)
        y_range = np.arange(y1, y2, main_field_size)
        
        for x in x_range:
            for y in y_range:
                main_field = [x, (x + main_field_size), y, (y + main_field_size)]
                
                if any(is_fully_covered(main_field, area) for area in care_areas):
                    if not any(is_overlap(main_field, field) for field in covered_areas):
                        main_fields.append([main_field_id] + main_field)
                        covered_areas.append(main_field)
                        main_field_id += 1
                        
    return main_fields

def place_sub_fields(main_fields, sub_field_size, care_areas):
    sub_fields = []
    sub_field_id = 0  
    
    for main_field in main_fields:
        _, x1, x2, y1, y2 = main_field
        
        x_range = np.arange(x1, x2, sub_field_size)
        y_range = np.arange(y1, y2, sub_field_size)
        
        for y in y_range:
            if (y // sub_field_size) % 2 == 0:  
                x_range_current = x_range
            else:  
                x_range_current = x_range[::-1]
                
            for x in x_range_current:
                sub_field = [sub_field_id, x, x + sub_field_size, y, y + sub_field_size, main_field[0]]
                
                # Check if the sub-field overlaps with any care area
                sub_field_rect = [sub_field[1], sub_field[2], sub_field[3], sub_field[4]]
                if any(is_overlap(sub_field_rect, care_area) for care_area in care_areas):
                    sub_fields.append(sub_field)
                    sub_field_id += 1
                
    return sub_fields

main_fields = place_main_fields(main_field_size, care_areas)
sub_fields = place_sub_fields(main_fields, sub_field_size, care_areas)

main_fields_sorted = sorted(main_fields, key=lambda field: field[1])  
sub_fields_sorted = sorted(sub_fields, key=lambda field: field[5])   

for i, field in enumerate(main_fields_sorted):
    field[0] = i  

for i, field in enumerate(sub_fields_sorted):
    field[0] = i

print(f"Number of main fields generated: {len(main_fields_sorted)}")
print(f"Number of sub-fields generated: {len(sub_fields_sorted)}")

main_fields_df = pd.DataFrame(main_fields_sorted, columns=['ID', 'Xmin', 'Xmax', 'Ymin', 'Ymax'])
main_fields_df.to_csv(mainfields_output_path, index=False)

sub_fields_df = pd.DataFrame(sub_fields_sorted, columns=['ID', 'Xmin', 'Xmax', 'Ymin', 'Ymax', 'MainFieldID'])
sub_fields_df.to_csv(subfields_output_path, index=False)

print("Saved to CSV files.")


Number of main fields generated: 169
Number of sub-fields generated: 67600
Saved to CSV files.
